# Un cas d'usage concret de régression avec MAPIE

## Contexte

La Californie est divisée en districts, pour chacun desquels une licence est requise pour y opérer en tant que professionnel de l'immobilier.* Tous les 10 ans, un recensement de la population est effectué, et les données, publiques, sont une mine d'or pour les promoteurs immobilier de l'état californien.

Ces données contiennent les informations suivantes :

| Variable   | Description |
|------------|------------|
| `MedHouseVal` | Valeur médiane des maisons d'un district donné (en centaines de milliers de dollars). |
| `MedInc`     | Revenu médian des ménages dans le district (en dizaines de milliers de dollars). |
| `HouseAge`   | Âge médian des maisons dans le district (en années). |
| `AveRooms`   | Nombre moyen de pièces par logement. |
| `AveBedrms`  | Nombre moyen de chambres par logement. |
| `Population` | Population totale du district. |
| `AveOccup`   | Nombre moyen de personnes par logement. |
| `Latitude`   | Latitude du centre du district. |
| `Longitude`  | Longitude du centre du district. |

Bill Smith, un promoteur sollicite votre aide. En effet, comme lors des recensements précédents, certains districts dits "sneaky"* ne souhaitent pas publier les données relatives à la valeur des maisons sur leur périmètre. Cette information est pourtant clé pour estimer s'il est intéressant d'effectuer les démarches pour acquérir une license dans un district donné.

Les jeux de données à votre disposition sont donc :
  - `X` et `y` : les données des districts ayant publié l'intégralité de leurs chiffres (`y` contient l'information `MedHouseVal`, et `X` les autres variables)
  - `X_sneaky` : les données des sneaky districts (sans l'information `MedHouseVal`)

Lors du précédent recensement, Mr. Smith a dû mobiliser ses équipes pendant plus d'une semaine pour estimer cette donnée manquante, district par district. Il souhaite cette fois-ci automatiser le processus.

Après plusieurs ateliers avec lui, vous avez défini ensemble un livrable sous forme de carte, affichant en vert, les sneaky districts dont la valeur médiane des maisons ne dépasse pas 150 000\$. Bill semble plutôt averse au risque, et préfère donc obtenir un nombre réduit de district dont on est sûrs.

*Note : ce cas d'usage est fictif, nous avons donc pris des libertés avec les lois réellement en vigueur en Californie !

## Plan d'action
1. Analyse exploratoire rapide des données
2. Entrainement d'un modèle GradientBoostingRegressor, car il est puissant et supporte la loss quantile (nous verrons l'intérêt de cette loss plus loin)
3. Conformalisation du modèle en utilisant la méthode ConformalizedQuantileRegressor
4. Prédire les intervalles de confiance sur le jeu de données X_sneaky pour la variable `MedHouseVal`
5. Affichage sur une carte des sneaky districts qui correspondent aux critères de Bill

# Imports et chargement du jeu de données

In [ ]:
import folium
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor
from scipy.stats import uniform, randint
from mapie.metrics import regression_coverage_score, regression_mean_width_score
from mapie_v1.regression import ConformalizedQuantileRegressor

In [ ]:
RANDOM_STATE = 42

In [ ]:
X = pd.read_csv("files_cas_d_usage/X.csv")
y = pd.read_csv("files_cas_d_usage/y.csv").squeeze("columns")
X_sneaky = pd.read_csv("files_cas_d_usage/X_sneaky.csv")

# Analyse exploratoire rapide des données (fournie)

## Valeurs nulles et doublons

Vérifions que le jeu de données X comprend des valeurs nulles ou des doublons

In [ ]:
print(X.info(), "\n")
print("Doublons :", X.duplicated().sum())

## Distribution des variables

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Histogram(x=y)
)
fig.update_layout(
    height=350,
    width=600,
    xaxis_title="Prix médian (100 000$)",
    title_text="Cible",
    font=dict(family="Computer Modern", size=18, color="#7f7f7f")
)

In [ ]:
rows, cols = 3, 3
fig = make_subplots(rows=rows, cols=cols, subplot_titles=X.columns)

for i, col in enumerate(X.columns):
    row = i // cols + 1
    col_num = i % cols + 1
    fig.add_trace(go.Histogram(y=X[col], name=col), row=row, col=col_num)

fig.update_layout(
    height=700,
    width=1200,
    title_text="Histograms",
    font=dict(family="Computer Modern", size=18, color="#7f7f7f")
)
fig.show()

## Corrélations entre les variables

In [ ]:
corr_matrix = X.corr()

fig = ff.create_annotated_heatmap(
    z=np.abs(corr_matrix.values),
    x=list(corr_matrix.columns),
    y=list(corr_matrix.index),
    annotation_text=corr_matrix.round(2).values,
    showscale=True
)
fig.update_layout(
    font=dict(family="Computer Modern", size=18, color="#7f7f7f")
)

# Entrainement d'un modèle prédictif

## Séparation des données d'entrainement et de conformalisation

Pour les méthodes dites "split" incluses dans MAPIE (`SplitConformalRegressor` et `ConformalizedQuantileRegressor`), les jeux d'entrainements et de conformalisation sont utilisés de la façon suivante :

![title](files_cas_d_usage/data-split.png)


**Exercice 1** : Lors du tutoriel précédent, nous avons utilisé le `ConformalizedQuantileRegressor` en laissant au soin de MAPIE l'entrainement du modèle. Cette fois-ci, nous allons donc utiliser le mode `prefit=True` avec un `GradientBoostingRegressor` entrainé par nos soins. On a donc besoin d'un jeu d'entrainement, d'un jeu de test, et, pour utiliser MAPIE, d'un jeu de conformalisation.
  - Créez un jeu d'entrainement (`X_train`, `y_train`) à partir de (`X`, `y`), en gardant 20% des données pour la partie test + conformalisation
  - À partir de ces données restantes, créez un jeu de conformalisation (`X_conformalize`, `y_conformalize`) et de test (`X_test`, `y_test`), de tailles égales

## Entrainement

On va utiliser un `GradientBoostingRegressor`, plutôt robuste aux variables corrélées et aux distributions asymétriques

**Exercice 2** :
  - Faites une recherche d'hyperparamètres sur un modèle sklearn de type `GradientBoostingRegressor`, avec la méthode `RandomizedSearchCV` (qui permet une validation croisée, ou "cross validation").
  - Ne dépassez pas 50 entrainements (ce nombre dépend de la grille de paramètres recherchés, et du nombre de plis ("folds").
  - Utilisez `loss="absolute_error"`

In [ ]:
param_distributions = {
    "n_estimators": randint(50, 500),
    "learning_rate": uniform(0.01, 0.3),
    "max_depth": randint(2, 10),
    "subsample": uniform(0.5, 0.5),
    "min_samples_split": randint(2, 20),
    "min_samples_leaf": randint(1, 20),
    "max_features": uniform(0.5, 0.5)
}

**Exercice 3** : pour le modèle le plus performant :
- Affichez ses hyperparamètres
- Sa MAE moyenne sur les jeux de validation croisés
 - Sa MAE sur le jeu de test
- L'interpretation de ces valeurs

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=y_test, 
    y=y_pred, 
    mode="markers", 
    name="Prédictions",
    marker=dict(color="blue", opacity=0.6)
))

min_val, max_val = np.min(y_test), np.max(y_test)
fig.add_trace(go.Scatter(
    x=[min_val, max_val], 
    y=[min_val, max_val], 
    mode="lines", 
    name="Ligne y = x", 
    line=dict(color="red", dash="dash")
))

fig.update_layout(
    title="Comparaison entre y_test et y_pred",
    xaxis_title="y_test (100 000$)",
    yaxis_title="y_pred (100 000$)",
    showlegend=True,
    font=dict(family="Computer Modern", size=18, color="#7f7f7f")
)

fig.show()

## Calcul des intervalles avec MAPIE

### Explication du fonctionnement du `ConformalizedQuantileRegressor`
Cette méthode a besoin non pas d'1 mais de 3 modèles : un modèle usuel entrainé à prédire la variable cible (notre modèle entrainé plus haut), et deux modèles entrainés à prédire respectivement les quantiles 5% et 95% de la variable cible. L'idée est que la variable cible se situe 90% du temps entre les prédictions de nos deux modèles quantiles à 5% et 95%.

Enfin, la phase de conformalisation permet de "calibrer" ces modèles pour fournir les garanties théoriques propres aux prédictions conformes.

![title](files_cas_d_usage/quantiles.png)

Nous devons donc avant d'utiliser MAPIE entrainer deux modèles supplémentaires correspondants aux quantiles haut et bas.

**Exercice 4** : entrainez les deux modèles quantile, avec les mêmes hyper paramètres que le modèle principal

In [ ]:
confidence_level = 0.90

**Exercice 5** : Par curiosité, intéressons-nous aux taux de couverture empirique sur le jeu de test que ces modèles fournissent, *avant* conformalisation avec MAPIE. Créez `y_pred_lower_quantile_before_mapie` et  `y_pred_upper_quantile_before_mapie`.

In [ ]:
print(f"Taux de couverture empirique avant MAPIE: {regression_coverage_score(y_test, y_pred_lower_quantile_before_mapie, y_pred_upper_quantile_before_mapie):.3f}")
print(f"Largeur moyenne des intervalles avant MAPIE : {round(regression_mean_width_score(y_pred_lower_quantile_before_mapie, y_pred_upper_quantile_before_mapie), 2)} (100k$)")

In [ ]:
df = pd.DataFrame((y_pred_upper_quantile_before_mapie - y_pred_lower_quantile_before_mapie), columns=["Largeur d'intervalles"])
fig = go.Figure()
fig.add_trace(
    go.Histogram(
        x=df["Largeur d'intervalles"]
    )
)
fig.update_layout(
    title_text="Distribution des largeurs d'intervalles (100 000$)",
    xaxis_title="Taille d'intervalle (100 000$)",
    font=dict(family="Computer Modern", size=18, color="#7f7f7f")
)
fig.show()

Que penser de ce taux de couverture empirique ? Utilisons maintenant MAPIE pour obtenir des garanties de couverture plus robustes :

**Exercice 6** : utilisez le `ConformalizedQuantileRegressor` en mode `prefit` pour prédire sur `X_test` les prédictions `y_preds_test` et les intervalles `y_pred_intervals_test`

Vérifions notre taux de couverture empirique sur le jeu de test, et calculons la taille moyenne de nos intervalles :

In [ ]:
y_pred_lower_quantile = y_pred_intervals_test[:, 0, 0]
y_pred_upper_quantile = y_pred_intervals_test[:, 1, 0]

print(f"Taux de couverture empirique : {regression_coverage_score(y_test, y_pred_lower_quantile, y_pred_upper_quantile):.3f}")
print(f"Largeur moyenne des intervalles : {round(regression_mean_width_score(y_pred_lower_quantile, y_pred_upper_quantile), 2)} (100 000$)")

intervals_before = (y_pred_upper_quantile_before_mapie - y_pred_lower_quantile_before_mapie)
intervals_after = (y_pred_upper_quantile - y_pred_lower_quantile)
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.15, 
                    subplot_titles=["Avant MAPIE", "Après MAPIE"])
fig.add_trace(go.Histogram(x=intervals_before, name="Avant MAPIE", nbinsx=20), row=1, col=1)
fig.add_trace(go.Histogram(x=intervals_after, name="Après MAPIE", nbinsx=20), row=2, col=1)
fig.update_layout(
    title_text="Distribution des largeurs d'intervalles (100 000$)",
    showlegend=False,
    font=dict(family="Computer Modern", size=18, color="#7f7f7f")
)
fig.show()

Le taux de couverture est-il satisfaisant ? Se fait-il au détriment d'un autre indicateur d'intérêt de notre problème ?

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=y_test[0::30], 
    y=y_pred[0::30],
    error_y=dict(
        type="data",
        array=intervals_after,
    ),
    mode="markers", 
    name="Prédictions",
    marker=dict(color="blue", opacity=0.6)
))

min_val, max_val = np.min(y_test), np.max(y_test)
fig.add_trace(go.Scatter(
    x=[min_val, max_val], 
    y=[min_val, max_val], 
    mode="lines", 
    name="Ligne y = x", 
    line=dict(color="red", dash="dash")
))

fig.update_layout(
    title="Comparaison entre y_test et y_pred avec barre d'erreurs (sur un sous-ensemble pour plus de lisibilité)",
    xaxis_title="y_test (100 000$)",
    yaxis_title="y_pred (100 000$)",
    showlegend=True,
    font=dict(family="Computer Modern", size=18, color="#7f7f7f")
)

fig.show()

# Production du livrable sous forme de carte

**Exercice 7** :
- Utilisez MAPIE pour calculer `y_preds_sneaky` et `y_pred_intervals_sneaky`
- Créez un array Numpy `green_sneaky_districts` contenant la liste des sneaky districts que nous voulons afficher (`True`) ou non (`False`) à partir de `y_pred_intervals_sneaky`
- Créez un Dataframe `map_data` contenant les colonnes `Latitude`, `Longitude`, et `Green district`
- Utilisez le code folium pour afficher la carte demandée par Bill

![title](files_cas_d_usage/green-districts.png)

In [ ]:
california_map = folium.Map(location=[36.7783, -119.4179], zoom_start=6, min_zoom=5)

for _, row in map_data.iterrows():
    if row["Green district"]:
        folium.CircleMarker(
            location=[row['Latitude'], row['Longitude']],
            radius=3,
            color="green",
            fill=True,
            fill_opacity=0.7,
        ).add_to(california_map)

n_sneaky_district_total = X_sneaky.shape[0]
n_green_district = map_data["Green district"].sum()
print("Nombre de sneaky districts total", n_sneaky_district_total)
print("Nombre de green districts", n_green_district)
print("Nombre de districts non affichés sur la carte", n_sneaky_district_total - n_green_district)

california_map

**Exercice 8** : nous avons choisi un taux de couverture de 90%. Quelle garantie statistique pouvons-nous communiquer à Bill à propos des green districts ?

# Retours de Bill

Bill est satisfait de ce premier livrable, et comprend bien la signification statistique de vos résultats. Il souhaite continuer à travailler avec vous pour affiner le travail fourni.

En particulier, il aimerait connaître les sneaky districts incertains mais prometteurs. C'est-à-dire ceux dont :
 - la valeur des maisons prédite par le modèle est inférieure à 150 000\$
 - mais dont la borne supérieure prédite par MAPIE dépasse ce plafond

Vous avez acculturé Bill aux limites de l'IA, et il comprend qu'il devra prendre une décision au cas par cas avec ses équipes pour ces districts.

Affinons le livrable en affichant en orange ces districts incertains.

![title](files_cas_d_usage/orange-districts.png)

**Exercice 9** :
- Créez un array Numpy `orange_sneaky_districts` contenant la liste des districts incertains que nous voulons afficher (`True`) ou non (`False`) à partir de `y_preds_sneaky` et  `y_pred_intervals_sneaky`.
- Ajouter au Dataframe `map_data` la colonne `Orange district`
- Affichez la nouvelle carte

In [ ]:
california_map = folium.Map(location=[36.7783, -119.4179], zoom_start=6, min_zoom=5)

for _, row in map_data.iterrows():
    if row["Green district"]:
        folium.CircleMarker(
            location=[row['Latitude'], row['Longitude']],
            radius=3,
            color="green",
            fill=True,
            fill_opacity=0.7,
        ).add_to(california_map)

for _, row in map_data.iterrows():
    if row["Orange district"]:
        folium.CircleMarker(
            location=[row['Latitude'], row['Longitude']],
            radius=1,
            color="orange",
            fill=True,
            fill_opacity=0.7,
        ).add_to(california_map)

n_orange_district = map_data["Orange district"].sum()
print("Nombre de sneaky districts total", n_sneaky_district_total)
print("Nombre de green districts", n_green_district)
print("Nombre de orange districts", n_orange_district)
print("Nombre de districts non affichés sur la carte", n_sneaky_district_total - n_green_district - n_orange_district)

california_map

Bravo ! Vous êtes désormais un expert des prédictions conformes ;)

# Pour aller plus loin

Quelques questions ouvertes pour prolonger la réflexion autour de ce cas d'usage :
1. Imaginons que Bill n'est pas adverse au risque mais plutôt tête brulée : Quels choix de niveau de confiance feriez-vous ? Quel choix de entre green et orange districts ?
2. Bill nous informe que le nombre total de chambre est un indicateur d'interêt dans ses calculs de rentabilité. On peut s'intéresser au taux de couverture conditionnel en fonction du nombre total de chambres (par exemple en découpant cette variable en catégories : 0-10, 10-20, 30-40, 40-50). Les taux de couverture correspondants à ces différentes catégories sont-ils homogènes ? Si non, quel moyen de remédiation pourrait-on utiliser ?
3. Notre problème s'apparente finalement à une classification binaire, voire ternaire : green, orange et red districts. Comment pourrait-on utiliser MAPIE dans ce cadre (spoiler : les outils de contrôle de risque)